In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
len(subjective_topics)

19

In [7]:
unique_topics = data_unfiltered['topics'].unique()

In [8]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [9]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [14]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [15]:
len(separate_topics)

45692

In [16]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [17]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [18]:
topics = dict_topic2img.keys()

### Get ground truth

In [19]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [20]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [21]:
#topic2english

In [22]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [23]:
len(english_dict_topic2imgGT['This is a photo of spices'])

419

In [24]:
Countries = set(data['country.name'])
len(Countries)

63

In [27]:
list_of_country_dicts = []  # list of ground truth imgs and text prompt mapping for each country; list of dict_topic2img for each country
list_of_topic2prompt_dict = []  # list of text prompt and topic mapping

for country in Countries:
    
    country_topics = ['This is a photo of ' + topic + ' from ' + country for topic in topics]
      
    topic2country = {}
    for topic, prompt in zip(topics, country_topics):
        topic2country[topic] = prompt
    list_of_topic2prompt_dict.append(topic2country)
    
    
    dict_countrytopic2imgGT = {}

    for topic in topics:
        key = topic2country[topic]
        dict_countrytopic2imgGT[key] = dict_topic2img[topic]
        
    list_of_country_dicts.append(dict_countrytopic2imgGT)

In [28]:
len(list_of_country_dicts)

63

## Plotting results

In [29]:
import pickle
with open("data/country_list_dict_topic2FNimg.pkl", 'rb') as f:
    lang_list_dict_topic2FNimg = pickle.load(f)

In [30]:
with open("data/country_list_dict_topic2F1.pkl", 'rb') as f:
    lang_list_dict_topic2F1 = pickle.load(f)

In [31]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [32]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


## Automating the analysis across 63 countries

In [55]:
from collections import Counter

all_topic_all_income_percountrydf = pd.DataFrame()
all_topic_all_country_perincomedf = pd.DataFrame()
language_recall_df = pd.DataFrame()
for lang_dict in tqdm.tqdm(lang_list_dict_topic2FNimg):
    
    #### forgotten images block (countrywise)
    
    list_all_countriesFN = []
    list_all_incomeFN = []
    dict_topic2FNCountry = {}
    dict_all_Countries_FN = {}
    
    for topic in lang_dict:
        country_suffix = topic.split(' ')[-1]
        if country_suffix in ['Faso', "d'Ivoire", 'Republic', 'Africa', 'Korea', 'Kingdom', 'States', 'Lanka']:
            country_suffix = topic.split(' ')[-2:]
            country_suffix = ' '.join(country_suffix)
        if country_suffix in ['Guinea']:
            country_suffix = topic.split(' ')[-3:]
            country_suffix = ' '.join(country_suffix)
        #dict_lang2translation = next(item for item in list_dict_lang2translation if topic.split(';')[1].strip() in list(item.values())[0])
        #dominant_lang = list(dict_lang2translation.keys())[0]
        list_imagesFN = lang_dict[topic]
        list_countriesFN = [dict_img2country[img] for img in list_imagesFN]
        list_incomepercountry = [dict_img2incomelevel[img] for img in list_imagesFN]
        
        for country in list_countriesFN:
            list_all_countriesFN.append(country)
        for income in list_incomepercountry:
            list_all_incomeFN.append(income)
        
        dict_topic2FNCountry[topic] = Counter(list_countriesFN)

    dict_all_Countries_FN = Counter(list_all_countriesFN)
    
    
    #### ground truth images block (countrywise)
    
    list_all_countriesGT = []
    list_all_incomeGT = []
    dict_topic2GTCountry = {}
    dict_all_Countries_GT = {}
    
    dict_lang_topic2img = next(item for item in list_of_country_dicts if topic in item.keys())
    
    for topic in dict_lang_topic2img:
        list_imagesGT = dict_lang_topic2img[topic]
        list_countriesGT = [dict_img2country[img] for img in list_imagesGT]
        list_incomepercountryGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for country in list_countriesGT:
            list_all_countriesGT.append(country)
        for income in list_incomepercountryGT:
            list_all_incomeGT.append(income)
            
        dict_topic2GTCountry[topic] = Counter(list_countriesGT)

    
    dict_all_Countries_GT = Counter(list_all_countriesGT)
    
    
    
    ### forgotten images block (income level wise)
    
    list_all_incomelevelsFN = []
    dict_topic2FNIncomelevels = {}
    dict_all_incomelevels_FN = {}
    
    mapped_dict_lang_topic2FNimg = next(item for item in lang_list_dict_topic2FNimg if topic in item.keys())
    
    for topic in mapped_dict_lang_topic2FNimg:
        list_imagesFN = mapped_dict_lang_topic2FNimg[topic]
        list_incomelevelsFN = [dict_img2incomelevel[img] for img in list_imagesFN]
        for incomelevel in list_incomelevelsFN:
            list_all_incomelevelsFN.append(incomelevel)

        dict_topic2FNIncomelevels[topic] = Counter(list_incomelevelsFN)


    dict_all_incomelevels_FN = Counter(list_all_incomelevelsFN)
    
    
    
    ### ground truth image block (income level wise)
    
    ### Income level ground truth

    list_all_incomelevelsGT = []
    dict_topic2GTIncomelevels = {}
    dict_all_incomelevels_GT = {}
    
    mapped_dict_lang_topic2img = next(item for item in list_of_country_dicts if topic in item.keys())
    for topic in mapped_dict_lang_topic2img:
        list_imagesGT = mapped_dict_lang_topic2img[topic]
        list_incomelevelsGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for incomelevel in list_incomelevelsGT: 
            list_all_incomelevelsGT.append(incomelevel)
        
        dict_topic2GTIncomelevels[topic] = Counter(list_incomelevelsGT)

    
    dict_all_incomelevels_GT = Counter(list_all_incomelevelsGT)

    
    #### dataframe building
    
    ################### all_topic_all_income_percountrydf ###########
    
    Forget_ratio_allincome_percountry = Counter({key : dict_all_Countries_FN[key] / dict_all_Countries_GT[key] for key in dict_all_Countries_FN})
    Forget_ratio_allincome_percountry_df = pd.DataFrame(Forget_ratio_allincome_percountry.items(),
                        columns = ['Country', 'Forget ratio'])
    Forget_ratio_allincome_percountry_df['country_suffix'] = [country_suffix for i in list(Forget_ratio_allincome_percountry_df['Forget ratio'])]
    Forget_ratio_allincome_percountry_df['Total images per country'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_GT)
    Forget_ratio_allincome_percountry_df['Forgotten images'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_FN)
    Forget_ratio_allincome_percountry_df['Recognized images'] = Forget_ratio_allincome_percountry_df['Total images per country'] - Forget_ratio_allincome_percountry_df['Forgotten images']
    Forget_ratio_allincome_percountry_df['Recall'] = Forget_ratio_allincome_percountry_df['Recognized images'] / Forget_ratio_allincome_percountry_df['Total images per country']
    
    all_topic_all_income_percountrydf = pd.concat([all_topic_all_income_percountrydf, Forget_ratio_allincome_percountry_df])
    
    
    ####################### alltopic_all_country_perincome ###################
    
    Forget_ratio_all_country_perincome = Counter({key : dict_all_incomelevels_FN[key] / dict_all_incomelevels_GT[key] for key in dict_all_incomelevels_FN})
    Forget_ratio_all_country_perincome_df = pd.DataFrame(Forget_ratio_all_country_perincome.items(), columns = ['Income level', 'Forget ratio',])
    Forget_ratio_all_country_perincome_df['Total images per income level'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_GT)
    Forget_ratio_all_country_perincome_df['Forgotten images'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_FN)
    #Forget_rationperincome_allcountry_df = Forget_ratio_percountry_allincome_df
    Forget_ratio_all_country_perincome_df['country_suffix'] = [country_suffix for i in list(Forget_ratio_all_country_perincome_df['Forget ratio'])]
    Forget_ratio_all_country_perincome_df['Recognized images'] = Forget_ratio_all_country_perincome_df['Total images per income level'] - Forget_ratio_all_country_perincome_df['Forgotten images']
    Forget_ratio_all_country_perincome_df['Recall'] = Forget_ratio_all_country_perincome_df['Recognized images'] / Forget_ratio_all_country_perincome_df['Total images per income level']
    
    all_topic_all_country_perincomedf = pd.concat([all_topic_all_country_perincomedf, Forget_ratio_all_country_perincome_df])
    
    
    
    ######################## all topic, per country, per income #####################
    
    Lang_FN_df = pd.DataFrame()
    Lang_FN_df['countries'] = list_all_countriesFN
    Lang_FN_df['country_suffix'] = [country_suffix for i in list_all_countriesFN]
    Lang_FN_df['income level'] = list_all_incomeFN
    Lang_forgotten = Lang_FN_df.groupby(['countries', 'income level']).count()
    Lang_forgotten.reset_index()
    
    lang_countries = [x[0] for x in Lang_forgotten.index]
    lang_incomelevel = [x[1] for x in Lang_forgotten.index]
    
    Lang_GT_df = pd.DataFrame()
    Lang_GT_df['countries'] = list_all_countriesGT
    Lang_GT_df['country_suffix'] = [country_suffix for i in list_all_countriesGT]
    Lang_GT_df['income level'] = list_all_incomeGT

    
    Lang_GT = Lang_GT_df.groupby(['countries', 'income level']).count()
    Lang_GT.reset_index()
    
    Lang_recall_df = pd.DataFrame()

    Lang_recall_df['country_suffix'] = [country_suffix for i in lang_countries]
    Lang_recall_df['countries'] = lang_countries
    Lang_recall_df['incomelevel'] = lang_incomelevel
    Lang_recall_df['forgotten'] = list(Lang_forgotten['country_suffix'])
    Lang_recall_df['ground truth'] =  list(Lang_GT['country_suffix'])
    Lang_recall_df['recall'] = 1 - (Lang_recall_df['forgotten']/ Lang_recall_df['ground truth'])
    
    language_recall_df = pd.concat([language_recall_df, Lang_recall_df])
    
    

all_topic_all_income_percountrydf.to_csv('country_all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf.to_csv('country_all_topic_all_country_perincomedf')
language_recall_df.to_csv('country_recall_df')

100%|██████████| 63/63 [00:07<00:00,  7.98it/s]


In [ ]:
import pandas as pd
country_all_topic_all_income_percountrydf = pd.read_csv('country_all_topic_all_income_percountrydf')
country_all_topic_all_country_perincomedf = pd.read_csv('country_all_topic_all_country_perincomedf')
country_recall_df = pd.read_csv('country_recall_df')

In [3]:
country_all_topic_all_country_perincomedf[country_all_topic_all_country_perincomedf['country_suffix'] == 'Cameroon']

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,country_suffix,Recognized images,Recall
88,0,poor,0.687202,11119,7641,Cameroon,3478,0.312798
89,1,lower-middle,0.711376,10997,7823,Cameroon,3174,0.288624
90,2,upper-middle,0.766818,11446,8777,Cameroon,2669,0.233182
91,3,rich,0.808362,12054,9744,Cameroon,2310,0.191638


In [12]:
country_recall_df

,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
0,0,Sweden,Austria,rich,183,284,0.355634
1,1,Sweden,Austria,upper-middle,81,144,0.437500
2,2,Sweden,Bangladesh,poor,629,753,0.164675
3,3,Sweden,Bolivia,lower-middle,177,245,0.277551
4,4,Sweden,Bolivia,poor,74,95,0.221053
...,...,...,...,...,...,...,...
8059,123,Jordan,United States,rich,517,1048,0.506679
8060,124,Jordan,United States,upper-middle,504,846,0.404255
8061,125,Jordan,Vietnam,lower-middle,396,491,0.193483
8062,126,Jordan,Vietnam,rich,96,145,0.337931


In [13]:
## Additional dataframe for lower levels only (images from poor and low-mid households)

low_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = country_recall_df[country_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['poor', 'lower-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'poor']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'lower-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['poor, lower-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()

low_lvl_language_recall_df.to_csv('country_low_lvl_recall_df')    

In [16]:
low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == 'Nigeria']

,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
70,70,Sweden,Nigeria,"poor, lower-middle",385,483,0.202899
198,70,Somalia,Nigeria,"poor, lower-middle",288,483,0.403727
326,70,Kazakhstan,Nigeria,"poor, lower-middle",357,483,0.260870
454,70,Mexico,Nigeria,"poor, lower-middle",341,483,0.293996
582,70,Denmark,Nigeria,"poor, lower-middle",377,483,0.219462
...,...,...,...,...,...,...,...
7494,70,Turkey,Nigeria,"poor, lower-middle",369,483,0.236025
7622,70,Egypt,Nigeria,"poor, lower-middle",336,483,0.304348
7750,70,Russia,Nigeria,"poor, lower-middle",370,483,0.233954
7878,70,France,Nigeria,"poor, lower-middle",363,483,0.248447


### By continent analysis for lower income images

In [18]:
countries_with_poor = ['Burkina Faso', 'Burundi', 'Liberia', 'Malawi', 'Rwanda', 'Somalia', 'Togo', 'Bangladesh', 'Bolivia', 'Cambodia', 'Cameroon', "Cote d'Ivoire", 'Haiti', 'India', 'Kenya', 'Myanmar', 'Nepal', 'Nigeria', 'Pakistan', 'Palestine', 'Papua New Guinea','Philippines', 'Tanzania', 'Tunisia', 'Zimbabwe', 'China', 'Colombia', 'Indonesia', 'Peru', 'South Africa', 'Thailand' ]
countries_with_low_mid = ['Ethiopia', 'Rwanda', 'Togo', 'Bolivia', 'Cambodia', 'Cameroon', 'Ghana','India', 'Jordan', 'Kenya', 'Mongolia', 'Myanmar', 'Nepal', 'Nigeria', 'Pakistan', 'Palestine', 'Philippines', 'Tanzania', 'Tunisia', 'Ukraine', 'Vietnam', 'Brazil', 'China', 'Colombia', 'Guatemala', 'Indonesia', 'Peru', 'Serbia', 'South Africa', 'Thailand', 'South Korea', 'United States']
countries_with_up_mid = ['Ethiopia','Cambodia', 'Cameroon', 'Egypt', 'India', 'Iran', 'Kyrgyzstan', 'Lebanon', 'Nepal', 'Nigeria', 'Pakistan', 'Philippines', 'Sri Lanka', 'Tanzania', 'Ukraine', 'Brazil', 'China', 'Colombia', 'Indonesia', 'Kazakhstan', 'Mexico', 'Russia', 'Serbia', 'South Africa', 'Thailand', 'Austria', 'France', 'Netherlands', 'South Korea', 'Sweden', 'United Kingdom', 'United States']
countries_with_rich = ['Ethiopia','India', 'Iran', 'Jordan', 'Kenya', 'Nepal', 'Nigeria', 'Pakistan', 'Ukraine', 'Vietnam', 'Brazil', 'China', 'Colombia', 'Indonesia', 'Mexico', 'Russia', 'Serbia', 'South Africa', 'Turkey', 'Austria', 'Canada', 'Czech Republic', 'Denmark', 'France', 'Italy', 'Netherlands', 'Romania', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']


In [34]:
African_countries_low = ['Burkina Faso', 'Burundi', 'Ethiopia', 'Liberia', 'Malawi', 'Rwanda', 'Somalia', 'Togo', 'Cameroon', "Cote d'Ivoire", 'Ghana', 'Kenya', 'Nigeria', 'Tanzania', 'Tunisia', 'Zimbabwe', 'South Africa']
American_countries_low = ['Bolivia', 'Haiti', 'Brazil', 'Colombia', 'Guatemala', 'Peru', 'United States']
Asian_countries_low = ['Bangladesh', 'Cambodia', 'India', 'Jordan', 'Mongolia', 'Myanmar', 'Nepal', 'Pakistan', 'Palestine', 'Papua New Guinea', 'Philippines', 'Vietnam', 'China', 'Indonesia', 'Thailand', 'South Korea']
European_countries_low = ['Ukraine', 'Serbia']
len(Asian_suffixes)

21

In [32]:
African_suffixes = ['Burkina Faso', 'Burundi', 'Ethiopia', 'Liberia', 'Malawi', 'Rwanda', 'Somalia', 'Togo', 'Cameroon', "Cote d'Ivoire", 'Egypt', 'Ghana', 'Kenya', 'Nigeria', 'Tanzania', 'Tunisia', 'Zimbabwe', 'South Africa']
American_suffixes = ['Bolivia', 'Haiti', 'Brazil', 'Colombia', 'Guatemala', 'Mexico', 'Peru', 'Canada', 'United States']
Asian_suffixes = ['Bangladesh', 'Cambodia', 'India', 'Iran', 'Jordan', 'Kyrgyzstan','Lebanon', 'Mongolia', 'Myanmar', 'Nepal', 'Pakistan', 'Palestine', 'Papua New Guinea', 'Philippines', 'Sri Lanka', 'Vietnam', 'China', 'Kazakhstan','Indonesia', 'Thailand', 'South Korea']
European_suffixes= ['Ukraine', 'Russia','Serbia', 'Turkey', 'Austria', 'Czech Republic', 'Denmark', 'France', 'Italy', 'Netherlands', 'Romania', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']

len(African_suffixes)+ len(American_suffixes) +len(Asian_suffixes)+len(European_suffixes)

63

In [76]:
country_suffix_dict = {}

for country in European_countries_low:
    c_c_list = []
    c_recall = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
    for i in European_suffixes:
        recall = c_recall[c_recall['country_suffix'] == i]['recall']
        c_c_list.append(recall.item())
    country_suffix_dict[country] = round(sum(c_c_list)/len(c_c_list), 3)

In [77]:
for i in country_suffix_dict.values():
    print(i)

0.443
0.359


### By continent analysis for higher income images

In [78]:
## Additional dataframe for higher levels only 

high_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = country_recall_df[country_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['rich', 'upper-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'rich']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'upper-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['rich, high-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        high_lvl_language_recall_df = pd.concat([high_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        high_lvl_language_recall_df = pd.concat([high_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()

In [79]:
high_lvl_language_recall_df

,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
0,0,Sweden,Austria,"rich, high-middle",264,428,0.383178
128,0,Somalia,Austria,"rich, high-middle",370,428,0.135514
256,0,Kazakhstan,Austria,"rich, high-middle",353,428,0.175234
384,0,Mexico,Austria,"rich, high-middle",341,428,0.203271
512,0,Denmark,Austria,"rich, high-middle",265,428,0.380841
...,...,...,...,...,...,...,...
7550,126,Turkey,Vietnam,rich,98,145,0.324138
7678,126,Egypt,Vietnam,rich,93,145,0.358621
7806,126,Russia,Vietnam,rich,84,145,0.420690
7934,126,France,Vietnam,rich,79,145,0.455172


In [80]:
high_lvl_language_recall_df['countries'].unique()

array(['Austria', 'Brazil', 'Cambodia', 'Cameroon', 'Canada', 'China',
       'Colombia', 'Czech Republic', 'Denmark', 'Egypt', 'Ethiopia',
       'France', 'India', 'Indonesia', 'Iran', 'Italy', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kyrgyzstan', 'Lebanon', 'Mexico', 'Nepal',
       'Netherlands', 'Nigeria', 'Pakistan', 'Philippines', 'Romania',
       'Russia', 'Serbia', 'South Africa', 'South Korea', 'Spain',
       'Sri Lanka', 'Sweden', 'Switzerland', 'Tanzania', 'Thailand',
       'Turkey', 'Ukraine', 'United Kingdom', 'United States', 'Vietnam'],
      dtype=object)

In [83]:
African_countries_high = ['Ethiopia', 'Cameroon', 'Egypt', 'Kenya', 'Nigeria', 'Tanzania', 'South Africa']
American_countries_high = ['Brazil', 'Colombia', 'Mexico', 'Canada', 'United States']
Asian_countries_high = ['Cambodia', 'India', 'Iran', 'Jordan', 'Kyrgyzstan','Lebanon', 'Nepal', 'Pakistan', 'Philippines', 'Sri Lanka', 'Vietnam', 'China', 'Kazakhstan','Indonesia', 'Thailand', 'South Korea']
European_countries_high= ['Ukraine', 'Russia', 'Serbia', 'Turkey', 'Austria', 'Czech Republic', 'Denmark', 'France', 'Italy', 'Netherlands', 'Romania', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']

len(African_countries_high)+ len(American_countries_high) +len(Asian_countries_high)+len(European_countries_high)

43

In [120]:
country_suffix_dict = {}

for country in European_countries_high:
    c_c_list = []
    c_recall = high_lvl_language_recall_df[high_lvl_language_recall_df['countries'] == country]
    for i in African_suffixes:
        recall = c_recall[c_recall['country_suffix'] == i]['recall']
        c_c_list.append(recall.item())
    country_suffix_dict[country] = round(sum(c_c_list)/len(c_c_list), 3)

In [122]:
for i in country_suffix_dict.keys():
    print(i)

Ukraine
Russia
Serbia
Turkey
Austria
Czech Republic
Denmark
France
Italy
Netherlands
Romania
Spain
Sweden
Switzerland
United Kingdom


In [13]:
## Additional dataframe for all levels per country_suffix

avg_all_lvl_language_recall_df = pd.DataFrame()

df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()

suffix = []
countri = []
recal = []
income = []

for country in df_countries:
    all_lvl = country_recall_df[country_recall_df['countries'] == country]
    
    for country_suff in df_countries:
        forgotten = []
        ground_truth = []
        #recall = []
        all_lvl_suff = all_lvl[all_lvl['country_suffix']==country_suff]
        
        recall = (all_lvl_suff.sum()['ground truth'] - all_lvl_suff.sum()['forgotten'])/ all_lvl_suff.sum()['ground truth']

        
        suffix.append(country_suff)
        countri.append(country)
        recal.append(recall)
        income.append('all')
           
avg_all_lvl_language_recall_df['country suffix'] = suffix
avg_all_lvl_language_recall_df['country'] = countri
avg_all_lvl_language_recall_df['recall'] = recal

avg_all_lvl_language_recall_df.to_csv('country_all_lvl_recall_df.csv') 

In [7]:
country_all_topic_all_income_percountrydf

,Unnamed: 0,Country,Forget ratio,country_suffix,Total images per country,Forgotten images,Recognized images,Recall
0,0,Burundi,0.879154,Sweden,331,291,40,0.120846
1,1,India,0.718933,Sweden,5625,4044,1581,0.281067
2,2,Somalia,0.944444,Sweden,54,51,3,0.055556
3,3,Papua New Guinea,0.888889,Sweden,99,88,11,0.111111
4,4,Rwanda,0.765306,Sweden,294,225,69,0.234694
...,...,...,...,...,...,...,...,...
3964,58,Togo,0.715385,Jordan,390,279,111,0.284615
3965,59,Liberia,0.772358,Jordan,246,190,56,0.227642
3966,60,Canada,0.562500,Jordan,48,27,21,0.437500
3967,61,Italy,0.691406,Jordan,256,177,79,0.308594


In [8]:
specific = country_recall_df[country_recall_df['countries'] == 'Colombia']
specific[specific['country_suffix'] == 'Colombia']


,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
1813,21,Colombia,Colombia,lower-middle,232,322,0.279503
1814,22,Colombia,Colombia,poor,212,338,0.372781
1815,23,Colombia,Colombia,rich,508,721,0.295423
1816,24,Colombia,Colombia,upper-middle,1106,1619,0.316862


In [9]:
country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == 'Burkina Faso'].sort_values('Recall')[-12:]

,Unnamed: 0,Country,Forget ratio,country_suffix,Total images per country,Forgotten images,Recognized images,Recall
351,36,Burkina Faso,0.674024,Rwanda,589,397,192,0.325976
3060,36,Burkina Faso,0.670628,Togo,589,395,194,0.329372
2430,36,Burkina Faso,0.668930,Zimbabwe,589,394,195,0.331070
603,36,Burkina Faso,0.662139,Ghana,589,390,199,0.337861
3248,35,Burkina Faso,0.655348,Tunisia,589,386,203,0.344652
1548,36,Burkina Faso,0.651952,Burkina Faso,589,384,205,0.348048
2745,36,Burkina Faso,0.634975,Liberia,589,374,215,0.365025
3438,36,Burkina Faso,0.634975,Nigeria,589,374,215,0.365025
855,36,Burkina Faso,0.626486,Burundi,589,369,220,0.373514
1421,35,Burkina Faso,0.614601,Cameroon,589,362,227,0.385399


In [10]:
specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == 'Burkina Faso']
recall = specific[specific['country_suffix'] == 'Burkina Faso']['Recall']
recall

1548    0.348048
Name: Recall, dtype: float64

## Query for table filling

In [3]:
## Query for country prompt recall for each country

Country_prompt_recall_list = []

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == country]
    recall = specific[specific['country_suffix'] == country]['Recall']
    recall = round(recall.item(), 3)
    Country_prompt_recall_list.append(recall)


In [19]:
## Query for country prompt recall for each country for lower income

Country_prompt_recall_list = []
country_prompt_dict = {}


unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()


for country in unique_countries:
    if country in df_countries:
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        recall = specific[specific['country_suffix'] == country]['recall']
        recall = round(recall.item(), 3)
        Country_prompt_recall_list.append(recall)
        country_prompt_dict[country] = recall

    else: 
        recall = '-'
        Country_prompt_recall_list.append(recall)
        country_prompt_dict[country] = recall
        

In [21]:
r_list = [i for i in Country_prompt_recall_list if i != '-']

In [20]:
for i in Country_prompt_recall_list:
    print(i)

-
0.467
0.306
0.319
0.348
0.535
0.477
0.374
-
0.467
0.327
0.232
-
-
-
0.446
-
0.439
0.496
0.241
0.431
0.437
-
-
0.448
-
0.456
-
-
0.366
0.294
-
0.412
0.488
0.466
-
0.435
0.476
0.491
0.394
0.365
0.41
-
-
0.446
0.367
0.333
0.276
0.5
-
-
-
-
0.505
0.396
0.436
0.345
-
0.577
-
0.463
0.515
0.289


In [22]:
for i in r_list:
    print(i)

0.467
0.306
0.319
0.348
0.535
0.477
0.374
0.467
0.327
0.232
0.446
0.439
0.496
0.241
0.431
0.437
0.448
0.456
0.366
0.294
0.412
0.488
0.466
0.435
0.476
0.491
0.394
0.365
0.41
0.446
0.367
0.333
0.276
0.5
0.505
0.396
0.436
0.345
0.577
0.463
0.515
0.289


In [16]:
## Query for country prompt recall for each country for low income only

country_prompt_top_dict = {}
country_prompt_worst_dict = {}
list_best_prompt = []
list_worst_prompt = []
bestest_prompt = []
rank_list = []

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    if country in df_countries:
    
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        rank = list(specific.sort_values('recall', ascending = False)['country_suffix']).index(country)
        best_country_prompt = list(specific.sort_values('recall', ascending = False)['country_suffix'][:5])
        best_country_recall = list(round(specific.sort_values('recall', ascending = False)['recall'][:1], 3))
        worst_country_prompt = list(specific.sort_values('recall', ascending = False)['country_suffix'][-5:])
        list_best_prompt.append(best_country_prompt)
        list_worst_prompt.append(worst_country_prompt[::-1])
        rank_list.append(rank+1)

        bestest_country_prompt = best_country_prompt[0], best_country_recall
        bestest_prompt.append(bestest_country_prompt) 
        country_prompt_top_dict[country] = best_country_prompt

        country_prompt_worst_dict[country] = worst_country_prompt[::-1]
        
    else:
        list_best_prompt.append(['-'])
        list_worst_prompt.append(['-'])
        rank_list.append('-')
        
        best_lang_prompt = '-'
        #bestest_lang_prompt = '-', '-'
        bestest_prompt.append('-') 
        country_prompt_top_dict[country] = best_lang_prompt

        country_prompt_worst_dict[country] = '-'
        

In [17]:
rank_list

['-',
 1,
 14,
 9,
 7,
 1,
 1,
 7,
 '-',
 1,
 5,
 37,
 '-',
 '-',
 '-',
 11,
 '-',
 17,
 1,
 11,
 1,
 2,
 '-',
 '-',
 4,
 '-',
 3,
 '-',
 '-',
 5,
 11,
 '-',
 6,
 1,
 1,
 '-',
 6,
 1,
 1,
 5,
 3,
 1,
 '-',
 '-',
 6,
 5,
 2,
 3,
 1,
 '-',
 '-',
 '-',
 '-',
 13,
 3,
 15,
 4,
 '-',
 2,
 '-',
 2,
 1,
 5]

In [18]:
## Query for country prompt recall for each country

country_prompt_top_dict = {}
country_prompt_worst_dict = {}
list_best_prompt = []
list_worst_prompt = []
bestest_prompt = []
rank_list = []

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    
    specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == country]
    rank = list(specific.sort_values('Recall', ascending = False)['country_suffix']).index(country)
    best_country_prompt = list(specific.sort_values('Recall', ascending = False)['country_suffix'][:5])
    best_country_recall = list(round(specific.sort_values('Recall', ascending = False)['Recall'][:1], 3))
    worst_country_prompt = list(specific.sort_values('Recall', ascending = False)['country_suffix'][-5:])
    list_best_prompt.append(best_country_prompt)
    list_worst_prompt.append(worst_country_prompt[::-1])
    rank_list.append(rank+1)
    
    bestest_country_prompt = best_country_prompt[0], best_country_recall
    bestest_prompt.append(bestest_country_prompt) 
    country_prompt_top_dict[country] = best_country_prompt
    
    country_prompt_worst_dict[country] = worst_country_prompt[::-1]

In [19]:
#rank_list

In [62]:
#country_prompt_worst_dict

In [20]:
##### recall_incomelevel_list = []
country_income_level_country_prompt_recall_dict = {}

for country in unique_countries:
    all_levels = country_recall_df[country_recall_df['countries'] == country]
    all_levels_specific = all_levels[all_levels['country_suffix'] == country]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    recallforincm = []
    for level in incm_lst:
        if level in list(all_levels_specific['incomelevel']):
            recall_val = round(all_levels_specific[all_levels_specific['incomelevel'] == level]['recall'].item(), 3)
            recallforincm.append(recall_val)
        else: 
            recall_val = '-'
            recallforincm.append(recall_val)
    country_income_level_country_prompt_recall_dict[country] = recallforincm

country_income_level_country_prompt_recall_dict

{'Austria': ['-', '-', 0.451, 0.327],
 'Bangladesh': [0.467, '-', '-', '-'],
 'Bolivia': [0.274, 0.318, '-', '-'],
 'Brazil': ['-', 0.319, 0.372, 0.327],
 'Burkina Faso': [0.348, '-', '-', '-'],
 'Burundi': [0.535, '-', '-', '-'],
 'Cambodia': [0.388, 0.525, 0.452, '-'],
 'Cameroon': [0.392, 0.365, 0.467, '-'],
 'Canada': ['-', '-', '-', 0.667],
 'China': [0.423, 0.472, 0.534, 0.487],
 'Colombia': [0.373, 0.28, 0.317, 0.295],
 "Cote d'Ivoire": [0.232, '-', '-', '-'],
 'Czech Republic': ['-', '-', '-', 0.397],
 'Denmark': ['-', '-', '-', 0.404],
 'Egypt': ['-', '-', 0.602, '-'],
 'Ethiopia': ['-', 0.446, 0.379, 0.467],
 'France': ['-', '-', 0.531, 0.439],
 'Ghana': ['-', 0.439, '-', '-'],
 'Guatemala': ['-', 0.496, '-', '-'],
 'Haiti': [0.241, '-', '-', '-'],
 'India': [0.294, 0.517, 0.55, 0.521],
 'Indonesia': [0.459, 0.428, 0.439, 0.552],
 'Iran': ['-', '-', 0.508, 0.462],
 'Italy': ['-', '-', '-', 0.473],
 'Jordan': ['-', 0.448, '-', 0.398],
 'Kazakhstan': ['-', '-', 0.505, '-'],
 'K

In [21]:
for i in country_income_level_country_prompt_recall_dict.values():
    print(i[3])

0.327
-
-
0.327
-
-
-
-
0.667
0.487
0.295
-
0.397
0.404
-
0.467
0.439
-
-
-
0.521
0.552
0.462
0.473
0.398
-
0.471
-
-
-
-
0.464
-
-
0.419
0.471
0.386
0.5
-
-
-
-
0.361
0.593
-
0.396
-
0.335
0.63
0.484
-
0.492
0.368
-
-
-
-
0.301
0.482
0.705
0.559
0.586
-


In [22]:
## Query for overall recall across countries and topics for each country-suffix prompt
participating_countries = list(low_lvl_language_recall_df['countries'].unique())


overall_recall_per_prompt = {}

for country in participating_countries:
    sub_df = country_all_topic_all_country_perincomedf[country_all_topic_all_country_perincomedf['country_suffix'] == country]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    orecallforincm = []
    for level in incm_lst:
        orecall_val = round(sub_df[sub_df['Income level'] == level]['Recall'].item(), 3)
        orecallforincm.append(orecall_val)
    overall_recall_per_prompt[country] = orecallforincm

overall_recall_per_prompt

{'Bangladesh': [0.271, 0.319, 0.266, 0.221],
 'Bolivia': [0.3, 0.318, 0.262, 0.223],
 'Brazil': [0.254, 0.303, 0.323, 0.303],
 'Burkina Faso': [0.327, 0.303, 0.227, 0.185],
 'Burundi': [0.331, 0.279, 0.197, 0.154],
 'Cambodia': [0.289, 0.288, 0.213, 0.172],
 'Cameroon': [0.313, 0.289, 0.233, 0.192],
 'China': [0.213, 0.34, 0.369, 0.319],
 'Colombia': [0.3, 0.324, 0.275, 0.232],
 "Cote d'Ivoire": [0.23, 0.296, 0.325, 0.302],
 'Ethiopia': [0.334, 0.313, 0.269, 0.227],
 'Ghana': [0.314, 0.294, 0.267, 0.233],
 'Guatemala': [0.269, 0.314, 0.277, 0.233],
 'Haiti': [0.296, 0.331, 0.307, 0.269],
 'India': [0.239, 0.31, 0.306, 0.278],
 'Indonesia': [0.266, 0.328, 0.303, 0.266],
 'Jordan': [0.222, 0.308, 0.376, 0.371],
 'Kenya': [0.296, 0.318, 0.283, 0.236],
 'Liberia': [0.327, 0.303, 0.249, 0.205],
 'Malawi': [0.301, 0.32, 0.286, 0.254],
 'Mongolia': [0.259, 0.326, 0.308, 0.256],
 'Myanmar': [0.263, 0.304, 0.241, 0.195],
 'Nepal': [0.274, 0.307, 0.253, 0.213],
 'Nigeria': [0.294, 0.286, 0.256, 

In [45]:
for i in overall_recall_per_prompt.values():
    print(i[0])


0.271
0.3
0.254
0.327
0.331
0.289
0.313
0.213
0.3
0.23
0.334
0.314
0.269
0.296
0.239
0.266
0.222
0.296
0.327
0.301
0.259
0.263
0.274
0.294
0.197
0.258
0.274
0.261
0.271
0.334
0.197
0.318
0.291
0.225
0.287
0.234
0.297
0.276
0.245
0.25
0.229
0.312


In [50]:
Rich = [i[3] for i in overall_recall_per_prompt.values()]
Rich

[0.221,
 0.223,
 0.303,
 0.185,
 0.154,
 0.172,
 0.192,
 0.319,
 0.232,
 0.302,
 0.227,
 0.233,
 0.233,
 0.269,
 0.278,
 0.266,
 0.371,
 0.236,
 0.205,
 0.254,
 0.256,
 0.195,
 0.213,
 0.223,
 0.289,
 0.317,
 0.235,
 0.287,
 0.295,
 0.204,
 0.354,
 0.209,
 0.269,
 0.314,
 0.228,
 0.256,
 0.252,
 0.284,
 0.323,
 0.391,
 0.294,
 0.242]

In [24]:
sub_df

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,country_suffix,Recognized images,Recall
152,0,poor,0.688281,11119,7653,Zimbabwe,3466,0.311719
153,1,lower-middle,0.689006,10997,7577,Zimbabwe,3420,0.310994
154,2,upper-middle,0.714922,11446,8183,Zimbabwe,3263,0.285078
155,3,rich,0.758089,12054,9138,Zimbabwe,2916,0.241911


In [25]:
unique_countries

['Austria',
 'Bangladesh',
 'Bolivia',
 'Brazil',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'China',
 'Colombia',
 "Cote d'Ivoire",
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Ethiopia',
 'France',
 'Ghana',
 'Guatemala',
 'Haiti',
 'India',
 'Indonesia',
 'Iran',
 'Italy',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kyrgyzstan',
 'Lebanon',
 'Liberia',
 'Malawi',
 'Mexico',
 'Mongolia',
 'Myanmar',
 'Nepal',
 'Netherlands',
 'Nigeria',
 'Pakistan',
 'Palestine',
 'Papua New Guinea',
 'Peru',
 'Philippines',
 'Romania',
 'Russia',
 'Rwanda',
 'Serbia',
 'Somalia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sri Lanka',
 'Sweden',
 'Switzerland',
 'Tanzania',
 'Thailand',
 'Togo',
 'Tunisia',
 'Turkey',
 'Ukraine',
 'United Kingdom',
 'United States',
 'Vietnam',
 'Zimbabwe']

In [26]:
poor = [round(0.224 - i[0], 2) for i in overall_recall_per_prompt.values()]
low_mid = [round(0.359 - i[1], 2) for i in overall_recall_per_prompt.values()]
up_mid = [round(0.456 - i[2], 2) for i in overall_recall_per_prompt.values()]
rich = [round(0.436 - i[3], 2) for i in overall_recall_per_prompt.values()]

In [27]:
round(sum(poor)/len(poor), 2)

-0.05

In [28]:
round(sum(low_mid)/len(low_mid), 2)

0.05

In [29]:
round(sum(up_mid)/len(up_mid), 2)

0.16

In [30]:
round(sum(rich)/len(rich), 2)

0.18

In [31]:
## all lower income per country

participating_countries = list(low_lvl_language_recall_df['countries'].unique())

In [32]:
len(participating_countries)

42

In [33]:
participating_countries

['Bangladesh',
 'Bolivia',
 'Brazil',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'China',
 'Colombia',
 "Cote d'Ivoire",
 'Ethiopia',
 'Ghana',
 'Guatemala',
 'Haiti',
 'India',
 'Indonesia',
 'Jordan',
 'Kenya',
 'Liberia',
 'Malawi',
 'Mongolia',
 'Myanmar',
 'Nepal',
 'Nigeria',
 'Pakistan',
 'Palestine',
 'Papua New Guinea',
 'Peru',
 'Philippines',
 'Rwanda',
 'Serbia',
 'Somalia',
 'South Africa',
 'South Korea',
 'Tanzania',
 'Thailand',
 'Togo',
 'Tunisia',
 'Ukraine',
 'United States',
 'Vietnam',
 'Zimbabwe']

In [ ]:
# Asia = 'Bangladesh','Cambodia','China','India','Indonesia','Jordan','Mongolia','Myanmar','Nepal','Pakistan','Palestine','Papua New Guinea',
# 'Philippines','South Korea','Thailand','Vietnam',
# America = 'Bolivia','Brazil','Colombia','Guatemala','Haiti','Peru','United States',
# Africa = 'Burkina Faso','Burundi','Cameroon',"Cote d'Ivoire",'Ethiopia','Kenya','Liberia','Malawi','Nigeria','Rwanda','Somalia','South Africa',
# 'Tanzania','Togo','Tunisia','Zimbabwe',
# Europe = 'Serbia','Ukraine'

In [34]:
participating_countries_rearr = ['Bangladesh','Cambodia','China','India','Indonesia','Jordan','Mongolia','Myanmar','Nepal','Pakistan','Palestine','Papua New Guinea',
'Philippines','South Korea','Thailand','Vietnam','Bolivia','Brazil','Colombia','Guatemala','Haiti','Peru','United States','Burkina Faso','Burundi','Cameroon',"Cote d'Ivoire",'Ethiopia','Ghana','Kenya','Liberia','Malawi','Nigeria','Rwanda','Somalia','South Africa',
'Tanzania','Togo','Tunisia','Zimbabwe','Serbia','Ukraine']

In [35]:
participating_countries_rearr_income = ['Burkina Faso','Burundi','Liberia','Ethiopia','Malawi','Rwanda','Somalia','Togo','Bangladesh','Bolivia','Cambodia','Cameroon',"Cote d'Ivoire",'Ghana','Kenya','Nigeria','India','Haiti','Jordan','Mongolia','Myanmar','Nepal','Pakistan','Papua New Guinea','Philippines',
'Tanzania','Tunisia','Palestine','Ukraine','Zimbabwe','Vietnam','Brazil','China','Indonesia','Thailand','Colombia','Guatemala','Peru','South Africa','Serbia','South Korea','United States']


In [36]:
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr_income:
    suff_recall = []
    for country_suff in participating_countries_rearr_income:
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        recall = round(specific[specific['country_suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

In [37]:
len(list_oflists)

42

In [38]:
suff2recallbycountry['Countries'] = participating_countries_rearr_income
    

In [39]:
len(suff2recallbycountry['Countries'])

42

In [42]:
import numpy as np

maplist = np.array(list_oflists)
np.save('inc_country_suff_maplist.npy', maplist)

In [43]:
suff2recallbycountrydf = pd.DataFrame.from_dict(suff2recallbycountry)
suff_recallbycountrydf=suff2recallbycountrydf.set_index('Countries')
suff_recallbycountrydf

,Burkina Faso,Burundi,Liberia,Ethiopia,Malawi,Rwanda,Somalia,Togo,Bangladesh,Bolivia,...,China,Indonesia,Thailand,Colombia,Guatemala,Peru,South Africa,Serbia,South Korea,United States
Countries,,,,,,,,,,,,,,,,,,,,,
Burkina Faso,0.348,0.508,0.329,0.459,0.324,0.449,0.278,0.513,0.412,0.235,...,0.246,0.374,0.263,0.224,0.243,0.253,0.245,0.184,0.287,0.113
Burundi,0.374,0.535,0.341,0.411,0.330,0.422,0.333,0.490,0.402,0.265,...,0.210,0.341,0.221,0.221,0.183,0.236,0.255,0.172,0.275,0.125
Liberia,0.365,0.498,0.366,0.451,0.350,0.480,0.167,0.533,0.349,0.265,...,0.188,0.367,0.208,0.259,0.278,0.279,0.299,0.200,0.287,0.162
Ethiopia,0.385,0.486,0.333,0.446,0.325,0.503,0.333,0.492,0.389,0.268,...,0.282,0.358,0.258,0.218,0.217,0.220,0.217,0.203,0.312,0.175
Malawi,0.287,0.447,0.313,0.479,0.294,0.412,0.204,0.469,0.378,0.229,...,0.237,0.382,0.287,0.230,0.217,0.193,0.269,0.184,0.325,0.188
Rwanda,0.326,0.474,0.386,0.449,0.324,0.446,0.222,0.487,0.394,0.282,...,0.277,0.410,0.275,0.256,0.252,0.226,0.285,0.214,0.362,0.162
Somalia,0.401,0.411,0.350,0.393,0.330,0.429,0.333,0.487,0.369,0.244,...,0.184,0.336,0.221,0.223,0.226,0.238,0.252,0.196,0.325,0.188
Togo,0.329,0.344,0.297,0.391,0.260,0.435,0.130,0.436,0.347,0.291,...,0.276,0.391,0.279,0.265,0.287,0.253,0.205,0.212,0.375,0.212
Bangladesh,0.121,0.208,0.297,0.308,0.136,0.279,0.167,0.259,0.467,0.218,...,0.295,0.397,0.367,0.202,0.165,0.189,0.160,0.172,0.287,0.125


In [44]:
suff_recallbycountrydf.to_csv('inc_suff_recallbycountrydf.csv')

In [ ]:
# repeat for all income
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr:
    suff_recall = []
    for country_suff in participating_countries_rearr:
        specific = country_recall_df[country_recall_df['countries'] == country]
        recall = round(specific[specific['country_suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

## All income

In [11]:
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr:
    suff_recall = []
    for country_suff in participating_countries_rearr:
        specific = avg_all_lvl_language_recall_df[avg_all_lvl_language_recall_df['country'] == country]
        recall = round(specific[specific['country suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

In [12]:
len(list_oflists)

42

In [13]:
suff2recallbycountry['Countries'] = participating_countries_rearr

In [14]:
import numpy as np

maplist = np.array(list_oflists)
np.save('allcountry_suff_maplist.npy', maplist)

In [15]:
allsuff2recallbycountrydf = pd.DataFrame.from_dict(suff2recallbycountry)
allsuff2recallbycountrydf=allsuff2recallbycountrydf.set_index('Countries')
allsuff2recallbycountrydf

,Bangladesh,Cambodia,China,India,Indonesia,Jordan,Mongolia,Myanmar,Nepal,Pakistan,...,Nigeria,Rwanda,Somalia,South Africa,Tanzania,Togo,Tunisia,Zimbabwe,Serbia,Ukraine
Countries,,,,,,,,,,,,,,,,,,,,,
Bangladesh,0.467,0.438,0.274,0.441,0.397,0.280,0.155,0.402,0.415,0.443,...,0.281,0.279,0.167,0.168,0.322,0.259,0.205,0.102,0.155,0.140
Cambodia,0.438,0.473,0.270,0.283,0.400,0.196,0.196,0.467,0.341,0.260,...,0.333,0.361,0.130,0.128,0.267,0.297,0.191,0.131,0.127,0.162
China,0.284,0.413,0.500,0.335,0.401,0.359,0.381,0.308,0.330,0.350,...,0.272,0.272,0.037,0.214,0.294,0.233,0.256,0.122,0.302,0.310
India,0.393,0.363,0.242,0.497,0.336,0.300,0.216,0.259,0.442,0.446,...,0.281,0.259,0.185,0.178,0.296,0.269,0.211,0.104,0.215,0.212
Indonesia,0.404,0.453,0.324,0.389,0.443,0.286,0.278,0.397,0.359,0.388,...,0.309,0.313,0.167,0.186,0.341,0.308,0.234,0.116,0.201,0.222
Jordan,0.214,0.315,0.294,0.307,0.341,0.435,0.268,0.175,0.298,0.363,...,0.277,0.303,0.148,0.274,0.301,0.285,0.319,0.200,0.353,0.386
Mongolia,0.324,0.458,0.434,0.290,0.416,0.259,0.412,0.383,0.363,0.271,...,0.282,0.310,0.148,0.190,0.294,0.303,0.229,0.160,0.235,0.288
Myanmar,0.409,0.473,0.348,0.337,0.392,0.194,0.278,0.488,0.349,0.307,...,0.281,0.310,0.093,0.152,0.301,0.272,0.172,0.098,0.140,0.168
Nepal,0.437,0.416,0.306,0.415,0.369,0.204,0.216,0.370,0.457,0.375,...,0.265,0.316,0.185,0.140,0.324,0.272,0.194,0.122,0.154,0.170


In [16]:
allsuff2recallbycountrydf.to_csv('allsuff2recallbycountrydf.csv')